# 2D Estimator

## Imports and Setup

In [1]:
import os
import src.bug_dataset
from src.cpm_model import CPM
from src.train_cpm_network import Train_CPM_Network
import numpy as np
import pandas as pd
from skimage import io
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
plt.ion()   # interactive mode

In [2]:
# Plotting Code
%matplotlib widget
import matplotlib.pyplot as plt

def plot_stick_bug(ax, points, vis, prediction=False):
    limb_ranges=[[0,7],[8,14],[15,21],[22,28],[29,35],[36,42],[43,49],[53,56],[59,62]]
    if len(points) < 62:
        return
    elif len(points) == 62:
        points = np.array(points).T
        for num in range(len(limb_ranges)):
            visible_limb = []
            for x in range(limb_ranges[num][0],limb_ranges[num][1]):
                if vis[x]== 1:
                    visible_limb.append(x) 
            if prediction:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb],'--', alpha=0.7, label='Prediction', color='red')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='X',s=10, color='red')
            else:
                line, = ax.plot(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], label='Actual', color='blue')
                ax.scatter(points[0][visible_limb], points[1][visible_limb], points[2][visible_limb], marker='o',s=4, color='blue')
        return ax, line

def set_axes_equal(ax):
    # workaround, as matplotlib's 3D plot has no option for equisised axes (10/2021)
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])


In [3]:
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))

In [4]:
reduceKeypoints = True
sungaya_dataset = src.bug_dataset.BugDataset(df=out_df,reduced=reduceKeypoints,
                             root_dir=target_dir,transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))

In [18]:
import cv2
sample = sungaya_dataset[154]
name = sample['file_name']
image = sample['image']
img=image.numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

name = sample['file_name']
keypoints = sample['key_points_2D']

keypoints = (sample['key_points_2D']*sungaya_dataset.std_2d)+sungaya_dataset.means_2d

heatmap = sample['heatmap'].numpy()

fig = plt.figure()
ax = fig.add_subplot()

ax.imshow(img.astype('uint8'))
heat =ax.imshow(cv2.resize(heatmap[0], (152,152)), alpha = 0.3,cmap='hot')
ax.scatter(keypoints[:,0], keypoints[:,1], marker='o', s=5,  color='b')

ax.set_xlim([0,152])
ax.set_ylim([152,0])
ax.set_aspect('equal')

plt.title(name)
fig.colorbar(heat)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
train_split = 0.7
valid_split = 0.1
train_size = int(train_split * len(sungaya_dataset))
valid_size = int(valid_split * len(sungaya_dataset))
test_size = len(sungaya_dataset) - (train_size+valid_size)
train_dataset,valid_dataset ,test_dataset = torch.utils.data.random_split(sungaya_dataset, [train_size, valid_size, test_size], generator=torch.Generator().manual_seed(42))

In [20]:
batch_size = 16

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [43]:
if reduceKeypoints:
    model = CPM(28)
    accurate_dist = torch.from_numpy(np.array([[2,2]]*28))

else:
    model = CPM(62)
    accurate_dist = torch.from_numpy(np.array([[1,1]]*62))


epochs = 1500
learning_rate = 8e-6
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, betas=(0.5, 0.999))
liftingtrainer = Train_CPM_Network(model,optimizer, accurate_dist, train_dataloader, valid_dataloader)
trained_model = liftingtrainer.run(epochs)

Using cuda device


CPM Model Epochs:   0%|          | 0/1500 [00:00<?, ?it/s]

Epoch 001:


Training Step:   0%|          | 0/143 [00:00<?, ?it/s]

Finished Epoch 001: | Train Loss: 1.17625 | Val Loss: 0.00000 | Train Acc: 0.000| Val Acc: 0.000
Epoch 002:


...

Training Step:   0%|          | 0/143 [00:00<?, ?it/s]

Finished Epoch 356: | Train Loss: 0.09387 | Val Loss: 0.00000 | Train Acc: 0.000| Val Acc: 0.000
Epoch 357:


Training Step:   0%|          | 0/143 [00:00<?, ?it/s]

Finished Epoch 357: | Train Loss: 0.09316 | Val Loss: 0.00000 | Train Acc: 0.000| Val Acc: 0.000
Epoch 358:


Training Step:   0%|          | 0/143 [00:00<?, ?it/s]

In [22]:
trained_model = liftingtrainer.model
trained_model.eval()
# Code get the first batch of results
with torch.no_grad():
    for data in train_dataloader:
        image = data['image']
        center = data['centermap']
        heatmap = data['heatmap']
        file_name = data['file_name']
        kp = data['key_points_2D']
        visib = data['visibility']
        input_var = image.to(liftingtrainer.device, dtype=torch.float)
        heatmap_var = heatmap.to(liftingtrainer.device, dtype=torch.float)
        centermap_var = center.to(liftingtrainer.device, dtype=torch.float)

        heat1, heat2, heat3, heat4, heat5, heat6 = trained_model(input_var, centermap_var)
        break

In [23]:
def get_kpt(maps, img_h = 152.0, img_w = 152.0):
        # maps (1,63,76,76)
        maps = maps.clone().cpu().data.numpy()
        map_6 = maps

        kpts = []
        for m in map_6[1:]:
            h, w = np.unravel_index(m.argmax(), m.shape)
            x = int(w * img_w / m.shape[1])
            y = int(h * img_h / m.shape[0])
            kpts.append([x,y])
        return np.array(kpts)

In [42]:
sample = 13
limb = 27

name = file_name[sample]

img = image[sample].numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

pred = heat6[sample].cpu().numpy()
pred_kp = get_kpt(heat6[sample])

acc = heatmap[sample].cpu().numpy()
acc_kp = get_kpt(heatmap[sample])

vis = visib[sample].cpu().numpy()
keypoints = (kp[sample].numpy()*sungaya_dataset.std_2d)+sungaya_dataset.means_2d

plt.figure()
plt.imshow(img)

# plt.scatter(pred_kp[limb,0]*vis, pred_kp[limb,1]*vis, marker='o', s=10,  color='r')
plt.scatter(acc_kp[limb,0]*vis, acc_kp[limb,1]*vis, marker='o', s=10,  color='b')

# plt.imshow(cv2.resize(pred[limb+1], (152,152)), alpha = .5,cmap='hot')
plt.imshow(cv2.resize(acc[limb+1], (152,152)), alpha = .5,cmap='hot')

plt.colorbar()
plt.title(name)
plt.show()
((abs(pred_kp- acc_kp)<np.array([1,1])).sum().item())/(28*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0.03571428571428571